### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Tidy up columns------------

school_data_complete = school_data_complete.rename(
    columns={"Student ID": "Student_ID",
            "budget": "Budget",
            "gender": "Gender",
            "grade": "Grade",
            "reading_score": "Reading_Score",
            "math_score": "Math_Score",
            "school_name": "School_Name",
            "type": "School_Type",
            "student_name": "Student_Name"})

In [3]:
#Using conditions to enable the use of SUM in the School Summary

#READING
#--------------------------
#set conditions to check if student passed reading
conditions = [
    (school_data_complete['Reading_Score'] >= 70),
    (school_data_complete['Reading_Score'] < 70)]

#set values to check if student passed reading
values = [1,0]

school_data_complete['Reading_Tier'] = np.select(conditions, values)

#MATH
#--------------------------
#set conditions to check if student passed Math
conditions = [
    (school_data_complete['Math_Score'] >= 70),
    (school_data_complete['Math_Score'] < 70)]

#set values to check if student passed Math
values = [1,0]

school_data_complete['Math_Tier'] = np.select(conditions, values)

#OVERALL CALCULATION to NUMBER
#-----------------------------
#Define Tier Sum
Tier_Sum = school_data_complete['Math_Tier'] +  school_data_complete['Reading_Tier']

#set conditions to check the sum of Math and Reading = 2
conditions = [
    (Tier_Sum == 2),
    (Tier_Sum < 2)]

#set values to check if student passed both
values = [1,0]

school_data_complete['Tier_Sum'] = np.select(conditions, values)

In [4]:
#Calculations for District Summary Dataframe

no_of_schools_list = list(school_data_complete.School_Name.value_counts()) 
no_of_schools = len(no_of_schools_list)
no_of_students = len(school_data_complete)
total_budget = sum(school_data_complete["Budget"].unique())
avg_math_score = school_data_complete["Math_Score"].mean()
avg_reading_score = school_data_complete["Reading_Score"].mean()
number_pass_math = school_data_complete['Student_ID'][school_data_complete['Math_Score'] >= 70].count()
number_pass_read = school_data_complete['Student_ID'][school_data_complete['Reading_Score'] >= 70].count()
percent_pass_math = (number_pass_math / no_of_students)*100
percent_pass_read = (number_pass_read / no_of_students)*100
overall_pass = school_data_complete[(school_data_complete['Reading_Score'] >= 70) &  
                (school_data_complete['Math_Score'] >= 70)]['Student_Name'].count()
percent_pass_both = (overall_pass / no_of_students)*100


In [5]:
#Create District Summary Dataframe

summary_df = pd.DataFrame({'Number of Schools': [no_of_schools],
                           'Number of Students': [no_of_students],
                           'Total Budget': [total_budget], 
                           'Average Maths Score': [avg_math_score], 
                           'Average Reading Score': [avg_reading_score],
                           '% Students passed Math': [percent_pass_math],
                           '% Students passed Reading': [percent_pass_read],
                           '% Students passed Overall': [percent_pass_both]
                          })
summary_df["Total Budget"] = summary_df["Total Budget"].map("${0:,.2f}".format)
summary_df["Number of Students"] = summary_df["Number of Students"].map("{:,}".format)

summary_df

,Number of Schools,Number of Students,Total Budget,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [6]:
#Create school table
school_data_df = school_data_complete.groupby(['School_Name', 'School_Type'])

#Calculations for School Summary Dataframe

#budget calculation - because the budget is listed as each line, 
#calculate the total / number of students per school
total_school_budget = school_data_df["Budget"].sum()
total_student_calc = school_data_df["Student_Name"].count()
school_budget = total_school_budget / total_student_calc

#student calculation
students_per_school = school_data_df["Student_ID"].count()

#per student calculation
budget_per_student = school_budget / students_per_school

#average scores calculation
maths_avg_per_school = school_data_df["Math_Score"].mean()
reading_avg_per_school = school_data_df["Reading_Score"].mean()

#passed per subject calculation
pass_math_per_school = school_data_df["Math_Tier"].sum()
pass_read_per_school = school_data_df["Reading_Tier"].sum()
pass_per_school = school_data_df["Tier_Sum"].sum()

#average pass numbers calculation
average_pass_read_per_school = (pass_read_per_school / total_student_calc)*100
average_pass_math_per_school = (pass_math_per_school / total_student_calc)*100
percent_pass_overall = (pass_per_school / total_student_calc)*100

In [7]:
#Create School Summary dataframe using variables defined above 

school_summary_df = pd.DataFrame({
                                  "No of Students": students_per_school,
                                  "Total Budget": school_budget,
                                  "No of Students": students_per_school,
                                  "Budget per Student": budget_per_student,
                                  "Average Maths Score": maths_avg_per_school,
                                  "Average Reading Score": reading_avg_per_school,
                                  "% Students passed Math": average_pass_math_per_school,
                                  "% Students passed Reading": average_pass_read_per_school,
                                  "% Students passed Overall": percent_pass_overall,
                                 })

school_summary_df["Total Budget"] = school_summary_df["Total Budget"].map("${0:,.2f}".format)
school_summary_df["No of Students"] = school_summary_df["No of Students"].map("{:,}".format)
school_summary_df["Budget per Student"] = school_summary_df["Budget per Student"].map("${:.2f}".format)

school_summary_df

,,No of Students,Total Budget,Budget per Student,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
School_Name,School_Type,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [8]:
#Sort descending by % Student passed Overall, print head to get 5 top performing schools

top_performing_df = school_summary_df.sort_values("% Students passed Overall", ascending=False)

top_performing_df.head()

,,No of Students,Total Budget,Budget per Student,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
School_Name,School_Type,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [9]:
#Sort ascending by % Student passed Overall, print head to get 5 bottom performing schools

bottom_performing_df = school_summary_df.sort_values("% Students passed Overall")

bottom_performing_df.head()

,,No of Students,Total Budget,Budget per Student,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
School_Name,School_Type,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
#Use loc to isolate grade dataframes - both reading and math 

ninthgrade_df = school_data_complete.loc[(school_data_complete["Grade"] == "9th"), 
                                         ['School_Name', 'Math_Score','Reading_Score']]
tenthgrade_df = school_data_complete.loc[(school_data_complete["Grade"] == "10th"), 
                                         ['School_Name','Math_Score','Reading_Score']]
eleventhgrade_df = school_data_complete.loc[(school_data_complete["Grade"] == "11th"), 
                                            ['School_Name','Math_Score','Reading_Score']]
twelfthgrade_df = school_data_complete.loc[(school_data_complete["Grade"] == "12th"), 
                                           ['School_Name','Math_Score','Reading_Score']]

In [11]:
#Create grade dataframes

ninthgrade_group_df = ninthgrade_df.groupby(['School_Name'])
tenthgrade_group_df = tenthgrade_df.groupby(['School_Name'])
eleventhgrade_group_df = eleventhgrade_df.groupby(['School_Name'])
twelfthgrade_group_df = twelfthgrade_df.groupby(['School_Name'])

#----------------------------

#Create ninth grade variables
ninth_maths_total = ninthgrade_group_df["Math_Score"].sum()
ninth_read_total = ninthgrade_group_df["Reading_Score"].sum()
ninth_total_student_calc = ninthgrade_group_df["Math_Score"].count()
ninth_maths_avg = (ninth_maths_total / ninth_total_student_calc)
ninth_read_avg = (ninth_read_total / ninth_total_student_calc)

#Create tenth grade variables
tenth_maths_total = tenthgrade_group_df["Math_Score"].sum()
tenth_read_total = tenthgrade_group_df["Reading_Score"].sum()
tenth_total_student_calc = tenthgrade_group_df["Math_Score"].count()
tenth_maths_avg = (tenth_maths_total / tenth_total_student_calc)
tenth_read_avg = (tenth_read_total / tenth_total_student_calc)

#Create eleventh grade variables
eleventh_maths_total = eleventhgrade_group_df["Math_Score"].sum()
eleventh_read_total = eleventhgrade_group_df["Reading_Score"].sum()
eleventh_total_student_calc = eleventhgrade_group_df["Math_Score"].count()
eleventh_maths_avg = (eleventh_maths_total / eleventh_total_student_calc)
eleventh_read_avg = (eleventh_read_total / eleventh_total_student_calc)

#Create twelfth grade variables
twelfth_maths_total = twelfthgrade_group_df["Math_Score"].sum()
twelfth_read_total = twelfthgrade_group_df["Reading_Score"].sum()
twelfth_total_student_calc = twelfthgrade_group_df["Math_Score"].count()
twelfth_maths_avg = (twelfth_maths_total / twelfth_total_student_calc)
twelfth_read_avg = (twelfth_read_total / twelfth_total_student_calc)

In [12]:
#Create Maths Score by Grade

maths_score_by_grade_df = pd.DataFrame({
                                    "9th": ninth_maths_avg,
                                    "10th": tenth_maths_avg,
                                    "11th": eleventh_maths_avg,
                                    "12th": twelfth_maths_avg,
                                  })
maths_score_by_grade_df

,9th,10th,11th,12th
School_Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [13]:
#Create Reading Score by Grade

read_score_by_grade_df = pd.DataFrame({
                                    "9th": ninth_read_avg,
                                    "10th": tenth_read_avg,
                                    "11th": eleventh_read_avg,
                                    "12th": twelfth_read_avg,
                                  })
read_score_by_grade_df

,9th,10th,11th,12th
School_Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [14]:
#Recreate school_summary_df without formatting to allow calculations

school_summary_df = pd.DataFrame({
                                  "No of Students": students_per_school,
                                  "Total Budget": school_budget,
                                  "No of Students": students_per_school,
                                  "Budget per Student": budget_per_student,
                                  "Average Maths Score": maths_avg_per_school,
                                  "Average Reading Score": reading_avg_per_school,
                                  "% Students passed Math": average_pass_math_per_school,
                                  "% Students passed Reading": average_pass_read_per_school,
                                  "% Students passed Overall": percent_pass_overall,
                                 })

In [15]:
#Scores by School Spending calculations


# Create the bins in which Data will be held
bins = [0, 584.9, 628.9, 644.9, 675]

# Names for the bins
studentbudget = ["<$584", "$585-629", "$630-644", "$645-675"]

# Add bins to School Summary table
school_summary_df["Budget-per-Student"] = pd.cut(school_summary_df["Budget per Student"], 
                                                 bins, labels=studentbudget, include_lowest=True)

#Create new DF grouped by bins
school_spending_df = school_summary_df.groupby("Budget-per-Student")

In [16]:
#Calculations for Scores by School Spending

#student calculation
students_per_budget = school_spending_df["No of Students"].sum()

#average scores calculation
maths_avg_per_budget = school_spending_df["Average Maths Score"].mean()
reading_avg_per_budget = school_spending_df["Average Reading Score"].mean()

#pass numbers calculation
pass_math_per_budget = school_spending_df["% Students passed Math"].mean()
pass_read_per_budget = school_spending_df["% Students passed Reading"].mean()
pass_per_budget = school_spending_df["% Students passed Overall"].mean()

In [17]:
#Create Scores by School Spending Dataframe

scores_by_spending_df = pd.DataFrame({
                                  "Average Maths Score": maths_avg_per_budget,
                                  "Average Reading Score": reading_avg_per_budget,
                                  "% Students passed Math": pass_math_per_budget,
                                  "% Students passed Reading": pass_read_per_budget,
                                  "% Students passed Overall": pass_per_budget
                                 })
scores_by_spending_df

,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
Budget-per-Student,,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [18]:
#Scores by School Size

# Create the bins in which Data will be held
bins = [0, 999, 1999, 5000]

# Names for the bins
school_size = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Add bins to School Summary table
school_summary_df["School Size"] = pd.cut(school_summary_df["No of Students"], 
                                          bins, labels=school_size, include_lowest=True)

#school_summary_df.head(20)

#Create new DF grouped by bins
school_size_df = school_summary_df.groupby("School Size")


In [19]:
#Calculations for Scores by Size

#student calculation
students_per_size = school_size_df["No of Students"].sum()

#average scores calculation
maths_avg_per_size = school_size_df["Average Maths Score"].mean()
reading_avg_per_size = school_size_df["Average Reading Score"].mean()

#pass numbers calculation
pass_math_per_size = school_size_df["% Students passed Math"].mean()
pass_read_per_size = school_size_df["% Students passed Reading"].mean()
pass_per_size = school_size_df["% Students passed Overall"].mean()

In [20]:
#Create Scores by School Size

scores_by_size_df = pd.DataFrame({
                                  "Average Maths Score": maths_avg_per_size,
                                  "Average Reading Score": reading_avg_per_size,
                                  "% Students passed Math": pass_math_per_size,
                                  "% Students passed Reading": pass_read_per_size,
                                  "% Students passed Overall": pass_per_size
                                 })
scores_by_size_df

,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [21]:
#Scores by School Type - use early school_summary_df
school_type_df = school_summary_df.groupby("School_Type")

In [22]:
#Calculations for Scores by Type

#student calculation
students_per_type = school_type_df["No of Students"].sum()

#average scores calculation
maths_avg_per_type = school_type_df["Average Maths Score"].mean()
reading_avg_per_type = school_type_df["Average Reading Score"].mean()

#pass numbers calculation
pass_math_per_type = school_type_df["% Students passed Math"].mean()
pass_read_per_type = school_type_df["% Students passed Reading"].mean()
pass_per_type = school_type_df["% Students passed Overall"].mean()

In [23]:
#Create Scores by School Type

school_type_df = pd.DataFrame({
                                  "Average Maths Score": maths_avg_per_type,
                                  "Average Reading Score": reading_avg_per_type,
                                  "% Students passed Math": pass_math_per_type,
                                  "% Students passed Reading": pass_read_per_type,
                                  "% Students passed Overall": pass_per_type
                                 })
school_type_df

,Average Maths Score,Average Reading Score,% Students passed Math,% Students passed Reading,% Students passed Overall
School_Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


* Conclusion

 - Based on the school budgets, there is an inverse correlation between school spend per student and results. This is confirmed in the list of 5 bottom performing schools being 5 of the top 7 schools for school spend per student.
 - Maths is the clear under performing department in District Schools - there is 16% average gap in pass rates between District & Charter Schools while there is a 27% gap in pass rates when comparing Maths results.
 - The size of schools appears to not have an impact on student results until it gets to >2000 students where there is a drop across the board.
 - Charter schools are generally smaller in both size and budget however produce far superior student results. 